## DSc Lab Assignment 8
##### Name: Indira Nandepu
##### Section: A
##### Roll no: 197155

In [1]:
# Importing the required modules
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


#### Helper functions used to build the neural network

In [2]:
def ReLU(z):
    # ReLU, on input x, returns 0 if x is non positive, and x itself if x is positive
    a = np.maximum(0, z)
    # returning the activation
    return a


# Initializes the parameters
def initparameters(layerSizes):
    parameters = {}
    for i in range(1, len(layerSizes)):
        parameters['W' + str(i)] = np.random.randn(layerSizes[i], layerSizes[i-1])*0.01
        parameters['B' + str(i)] = np.random.randn(layerSizes[i],1)*0.01
    return parameters


# Performs Forward propagation
def forwardProp(X_train, parameters):
    layers = len(parameters)//2
    values = {}
    for i in range(1, layers+1):
        if i==1:
            values['Z' + str(i)] = np.dot(parameters['W' + str(i)], X_train) + parameters['B' + str(i)]
            values['A' + str(i)] = ReLU(values['Z' + str(i)])
        else:
            values['Z' + str(i)] = np.dot(parameters['W' + str(i)], values['A' + str(i-1)]) + parameters['B' + str(i)]
            if i==layers:
                values['A' + str(i)] = values['Z' + str(i)]
            else:
                values['A' + str(i)] = ReLU(values['Z' + str(i)])
    return values


# Cost computation
def computeCost(vals, Y_train):
    layers = len(vals)//2
    Y_pred = vals['A' + str(layers)]
    cost = 1/(2*len(Y_train)) * np.sum(np.square(Y_pred - Y_train))
    return cost


# Backpropagation
def backwardProp(parameters, vals, X_train, Y_train):
    layers = len(parameters)//2
    m = len(Y_train)
    grads = {}
    for i in range(layers,0,-1):
        if i==layers:
            dA = 1/m * (vals['A' + str(i)] - Y_train)
            dZ = dA
        else:
            dA = np.dot(parameters['W' + str(i+1)].T, dZ)
            dZ = np.multiply(dA, np.where(vals['A' + str(i)]>=0, 1, 0))
        if i==1:
            grads['W' + str(i)] = 1/m * np.dot(dZ, X_train.T)
            grads['B' + str(i)] = 1/m * np.sum(dZ, axis=1, keepdims=True)
        else:
            grads['W' + str(i)] = 1/m * np.dot(dZ,vals['A' + str(i-1)].T)
            grads['B' + str(i)] = 1/m * np.sum(dZ, axis=1, keepdims=True)
    return grads


# Updates the parameters
def updparameters(parameters, gradients, alpha):
    layers = len(parameters)//2
    parameters_updated = {}
    for i in range(1,layers+1):
        parameters_updated['W' + str(i)] = parameters['W' + str(i)] - alpha * gradients['W' + str(i)]
        parameters_updated['B' + str(i)] = parameters['B' + str(i)] - alpha * gradients['B' + str(i)]
    return parameters_updated


# Training the model
def model(X_train, Y_train, layer_sizes, num_iters, learning_rate):
    parameters = initparameters(layer_sizes)
    for i in range(num_iters):
        values = forwardProp(X_train.T, parameters)
        cost = computeCost(values, Y_train.T)
        grads = backwardProp(parameters, values,X_train.T, Y_train.T)
        parameters = updparameters(parameters, grads, learning_rate)
        print('Cost at iteration ' + str(i+1) + ' : ' + str(cost) + '\n')
    return parameters


# Predicting the values
def predict(X, params):
    vals = forwardProp(X.T, params)
    predictions = vals['A' + str(len(vals)//2)].T
    return predictions


# Computing the accuracy
def compute_accuracy(X_train, X_test, Y_train, Y_test, params):
    trainVals = forwardProp(X_train.T, params)
    testVals = forwardProp(X_test.T, params)
    trainAcc = np.sqrt(mean_squared_error(Y_train, trainVals['A' + str(len(layer_sizes)-1)].T))
    testAcc = np.sqrt(mean_squared_error(Y_test, testVals['A' + str(len(layer_sizes)-1)].T))
    return trainAcc, testAcc

#### 1. Making predictions to the boston housing dataset using the Neural network built from scratch

In [3]:
# Initialising the hyper-parameters
layer_sizes = [13, 8, 4, 1]                                                       #set layer sizes, do not change the size of the first and last layer 
num_iters = 50                                                                 #set number of iterations over the training set(also known as epochs in batch gradient descent context)
learning_rate = 0.05

# loading the csv file
boston_dataset =  pd.read_csv('boston.csv')

n=boston_dataset.shape[1]
m=boston_dataset.shape[0]

x_train=np.array(boston_dataset.iloc[:int(0.8*m),:n-1])
y_train=np.array(boston_dataset.iloc[:int(0.8*m),-1])  
x_test=np.array(boston_dataset.iloc[int(0.8*m):,:n-1])
y_test=np.array(boston_dataset.iloc[int(0.8*m):,-1])

params = model(x_train, y_train, layer_sizes, num_iters, learning_rate)

y_predict = predict(x_test,params)

import math
error = math.sqrt(np.mean((y_test - y_predict)**2))

print("RMS error of the model=> ", error)


Cost at iteration 1 : 334.7289955854144

Cost at iteration 2 : 334.65401575230965

Cost at iteration 3 : 334.57882818968

Cost at iteration 4 : 334.50340246347645

Cost at iteration 5 : 334.4277148754548

Cost at iteration 6 : 334.35175527055253

Cost at iteration 7 : 334.27547886580226

Cost at iteration 8 : 334.1988389982693

Cost at iteration 9 : 334.12178876786265

Cost at iteration 10 : 334.04424399169454

Cost at iteration 11 : 333.96607195825453

Cost at iteration 12 : 333.88714411624113

Cost at iteration 13 : 333.8073371124016

Cost at iteration 14 : 333.7264743108448

Cost at iteration 15 : 333.64433911620864

Cost at iteration 16 : 333.5606785218045

Cost at iteration 17 : 333.4751911726136

Cost at iteration 18 : 333.38750734169577

Cost at iteration 19 : 333.297159003953

Cost at iteration 20 : 333.20357573014786

Cost at iteration 21 : 333.10589717654625

Cost at iteration 22 : 333.0021694128727

Cost at iteration 23 : 332.8888842819651

Cost at iteration 24 : 332.7644595

#### 2. Making predictions to the seeds dataset using the Neural network built from scratch

In [4]:
# Initialise the hyperparameters
num_iters = 1000                                                                  #set number of iterations over the training set(also known as epochs in batch gradient descent context)
learning_rate = 0.05
layer_sizes=[7,6,4,3]

df =  pd.read_csv('seeds.csv')
n=df.shape[1]
m=df.shape[0]

x_train=np.array(df.iloc[:int(0.8*m),:n-1])
y_train=np.array(df.iloc[:int(0.8*m),-1])  
x_test=np.array(df.iloc[int(0.8*m):,:n-1])
y_test=np.array(df.iloc[int(0.8*m):,-1]) 
params = model(x_train, y_train, layer_sizes, num_iters, learning_rate)
y_predict = predict(x_test,params)

print(y_predict)

Cost at iteration 1 : 5.385114467445681

Cost at iteration 2 : 5.382354525028408

Cost at iteration 3 : 5.379596232706571

Cost at iteration 4 : 5.3768395892651775

Cost at iteration 5 : 5.374084596510653

Cost at iteration 6 : 5.371331253435655

Cost at iteration 7 : 5.36857955733868

Cost at iteration 8 : 5.365829505167856

Cost at iteration 9 : 5.363081094660069

Cost at iteration 10 : 5.360334328247454

Cost at iteration 11 : 5.357589204675126

Cost at iteration 12 : 5.354845722604068

Cost at iteration 13 : 5.352103882884311

Cost at iteration 14 : 5.349363684818263

Cost at iteration 15 : 5.346625127404142

Cost at iteration 16 : 5.343888208550597

Cost at iteration 17 : 5.341152927640914

Cost at iteration 18 : 5.338419283257103

Cost at iteration 19 : 5.33568727365674

Cost at iteration 20 : 5.332956899160859

Cost at iteration 21 : 5.330228154902862

Cost at iteration 22 : 5.327501043349976

Cost at iteration 23 : 5.324775564465555

Cost at iteration 24 : 5.322051717253389

Co